### 2.9

__Implement a performance-measuring environment simulator for the vacuum-cleaner world depicted in Figure 2.2 and specified on page 38. Your implementation should be modular so that the sensors, actuators, and environment characteristics (size, shape, dirt placement, etc.) can be changed easily. (_Note:_ for some choices of programming language and operating system there are already implementations in the [online code repository](http://aima.cs.berkeley.edu/code.html).)__

The world in Figure 2.2 has two squares, "A" and "B". I have implemented this as `vacuum_cleaner_world.environments.SimpleVacuumWorld`.

The specifications from page 38 are as follow:

* The performance measure awards one point for each clean square at each time step, over a  "lifetime" of 1000 time steps.
* The geography of the environment is known _a priori_ but the dirt distribution and the initial location of the agent are not. Clean squares stay clean and sucking cleans the current square. The _Left_ and _Right_ actions move the agent left and right except when this would take the agent outside the environment, in which case the agent remains where it is.
* The only available actions are _Left_,  _Right_, and _Suck_.
* The agent correctly perceives its location and whether that location contains dirt.

In [1]:
from vacuum_cleaner_world.environment import SimpleVacuumWorld

### 2.10
__Consider a modified version of the vacuum environment in Exercise 2.9, in which the agent is penalized one point for each movement.__

__a. Can a simple reflex agent be perfectly rational for this environment? Explain.__

A simple reflex agent cannot be perfectly rational. Since clearly a reflex agent that returns the action `Clean` when the dirt sensor informs it that there is dirt in its location will do better than one that moves, the action for percepts `[A, Dirt]` and `[B, Dirt]` will be `Clean`. For `[A, No Dirt]` a simple reflex agent can either return `Clean`, `Left`, or `Right`.

`Clean` or `Left` will cause the agent to stay in the same place for its lifetime. The expected performance of this agent will be poor, if square B is expected to have dirt. However, a reflex agent which returns `Right` for `[A, No Dirt]` and `Left` for `[B, No Dirt]` will oscillate back and forth once both squares are cleaned. An agent that stays still after visiting both squares would perform better, but no set of condition-action rules,  and therefore no simple reflex agent, can implement such an agent function.

In [2]:
from vacuum_cleaner_world.agents import SimpleReflexAgent

In [3]:
def run_trials(trials, env, AgentObject, agent_name, **kwargs):
    scores = [env.simulate(AgentObject, **kwargs) for _ in range(trials)]
    
    print('{} had an average performance of {:.2f} over {} trials'.format(agent_name,
                                                                          sum(scores)/len(scores), 
                                                                          trials))
    
    return scores

In [4]:
env = SimpleVacuumWorld(move_penalty=True)

run_trials(100, env, SimpleReflexAgent, 'Simple Reflex Agent');

Simple Reflex Agent had an average performance of 1000.35 over 100 trials


It would be sufficient to show that other agents can do better than this to prove that `SimpleReflexAgent` is not rational.

__b. What about a reflex agent with state? Design such an agent.__

Since it takes just 1 movement to visit every square, a reflex agent in this environment only needs to keep track of how many moves it has made, and return `NoOp` if this is equal to 1.

The stateful reflex agent is implemented as `vacuum_cleaner_world.agents.StatefulReflexAgent`

The maximum scores attainable are:

__1999__ if there is initially no dirt, or dirt in the agent's starting square...

__1998__ if there is dirt in the opposite square...

__1997__ if there is initially dirt in both squares...

Assuming that the first score taken is after the agent takes its first action.

By manually setting the initial dirt and agent location, I show that the `StatefulReflexAgent` is rational.

In [5]:
from vacuum_cleaner_world.agents import StatefulReflexAgent

In [6]:
def simulate_all_dirt_agent_configs(AgentObject, **kwargs):
    dirt_inits = ['dirty', [0, 1], [1, 0], [0, 1], [1, 0], 'clean']
    agent_inits = [None, 'A', 'B', 'B', 'A', None]
    simulation_names = ['Both Squares Dirty', 
                        'Agent in A, Dirt in B',
                        'Agent in B, Dirt in A',
                        'Agent in B, Dirt in B',
                        'Agent in A, Dirt in A',
                        'Both Squares Clean']
    
    scores = []
    for dirt_init, agent_init, name in zip(dirt_inits, agent_inits, simulation_names):
        env = SimpleVacuumWorld(dirt_init=dirt_init, init_loc=agent_init, **kwargs)
        score = env.simulate(AgentObject)
        scores.append(score)
        
    return simulation_names, scores

In [7]:
names, scores = simulate_all_dirt_agent_configs(StatefulReflexAgent, move_penalty=True)
for name, score in zip(names, scores):
    print('{}: {}'.format(name, score))

Both Squares Dirty: 1997
Agent in A, Dirt in B: 1998
Agent in B, Dirt in A: 1998
Agent in B, Dirt in B: 1999
Agent in A, Dirt in A: 1999
Both Squares Clean: 1999


__c. How do your answers to *a* and *b* change if the agent's percepts give it the clean/dirty status of every square in the environment?__

A simple reflex agent can be perfectly rational in this environment. Therefore, a reflex agent with state can also be perfectly rational, since it can have the same set of condition-action rules as the simple reflex agent, and maintain a state variable that does not do anything (e.g. always being equal to 0).

I implement a simple reflex agent with access to all percepts as `vacuum_cleaner_world.agents.FullInfoReflexAgent`. Its condition action rules are:

| Percepts                    | Action | 
| --------------------------- |:------:|
| (`A`, `Dirt A`, `Dirt B`)   | `Suck` | 
| (`B`, `Dirt A`, `Dirt B`)   | `Suck` | 
| (`B`, `Clean A`, `Dirt B`)  | `Suck` | 
| (`A`, `Dirt A`, `Clean B`)  | `Suck` | 
| (`A`, `Clean A`, `Dirt B`)  | `Right`| 
| (`B`, `Dirt A`, `Clean B`)  | `Left` | 
| (`A`, `Clean A`, `Clean B`) | `NoOp` | 
| (`B`, `Clean A`, `Clean B`) | `NoOp` | 

In [8]:
from vacuum_cleaner_world.agents import FullInfoReflexAgent

In [9]:
names, scores = simulate_all_dirt_agent_configs(FullInfoReflexAgent, 
                                                move_penalty=True, 
                                                perfect_information=True)

for name, score in zip(names, scores):
    print('{}: {}'.format(name, score))

Both Squares Dirty: 1997
Agent in A, Dirt in B: 1998
Agent in B, Dirt in A: 1998
Agent in B, Dirt in B: 2000
Agent in A, Dirt in A: 2000
Both Squares Clean: 2000


Given that this agent receives more percepts, its standard of rationality is higher. It can achieve a perfect score of 2000 since it can know with certainty that there is no need to visit the other square if there is no dirt there initially. Note that the reflex agent with state from __b__ is still rational, because given its percepts, its expected performance measure is low if it does not visit each square once since there is some probability of it finding dirt in the square it did not start in.

### 2.11 

__Consider a modified version of the vacuum environment in Exercise 2.9, in which the geography of the environment - its extent, boundaries, and obstacles - is unknown, as is the initial dirt configuration. (The agent can go *Up* and *Down* as well as *Left* and *Right*.)__

__a. Can a simple reflex agent be perfectly rational for this environment? Explain.__

A simple reflex agent cannot be rational, because it cannot maintain a model of the environment, which would be necessary for it to know and remember where squares are situated in the initially unknown geography. Even if it were to perceive that it was in square `A`, this information would not mean anything, because the agent would not know which squares, if any, would be next to square `A`. Since the geography of the environment is unknown, the only percept information a reflex agent can act on is that of whether there is dirt or no dirt in its current square. Therefore, the only implementable agent functions are those where the agent cleans, and then moves in a single direction whenever there is no dirt in its current square.

In [10]:
from vacuum_cleaner_world.environment import UnknownVacuumWorld
from vacuum_cleaner_world.agents import SimpleReflexAgentUnknown

In [11]:
geography = [['AA', 'AB', 'AC', None],
             ['BA', 'BB', 'BC', 'BD'],
             ['CA', None, 'CC', 'CD'],
             ['DA', 'DB', 'DC', 'DD']]

env = UnknownVacuumWorld(geography=geography)

In [12]:
env.display_geography()

|=||=||=|   
|=||=||=||=|
|=|   |=||=|
|=||=||=||=|


In [13]:
names = [' (Traveling Up)', ' (Traveling Down)', ' (Traveling Left)', ' (Traveling Right)']
directions = ['Up', 'Down', 'Left', 'Right']

for name, direction in zip(names, directions):
    scores = run_trials(100, env, SimpleReflexAgentUnknown, 
                        "Simple Reflex Agent"+name, direction=direction)
    print('First 10 scores:', scores[:10], '\n')

Simple Reflex Agent (Traveling Up) had an average performance of 7988.55 over 100 trials
First 10 scores: [9000, 7000, 6000, 6996, 8000, 8999, 8998, 8996, 6999, 6000] 

Simple Reflex Agent (Traveling Down) had an average performance of 8238.67 over 100 trials
First 10 scores: [7000, 8000, 8998, 7000, 9000, 7000, 6998, 6000, 6998, 10000] 

Simple Reflex Agent (Traveling Left) had an average performance of 7918.74 over 100 trials
First 10 scores: [8000, 7000, 7999, 6000, 3998, 9000, 4000, 8000, 10998, 8000] 

Simple Reflex Agent (Traveling Right) had an average performance of 8158.70 over 100 trials
First 10 scores: [6994, 8000, 7000, 9998, 7996, 2000, 5999, 8997, 2000, 10000] 



__b. Can a simple reflex agent with a _randomized_ agent function outperform a simple reflex agent? Design such an agent and measure its performance on several environments.__

An agent that moves randomly when its current square is clean will outperform an agent that moves in a single direction in most environments, excepting those with the geography of a straight line where the simple reflex agent happens to start at one end and has condition action rules that cause it to move towards the other end after cleaning. Outside of these special cases, random movements will explore the environment more effectively, if not particularly efficiently overall.

My implementation initializes each square with a 50% chance of containing dirt.

In [14]:
from vacuum_cleaner_world.agents import RandomizedReflexAgent

_Sample Environment 1_

In [15]:
env.display_geography()

|=||=||=|   
|=||=||=||=|
|=|   |=||=|
|=||=||=||=|


In [16]:
scores = run_trials(100, env, RandomizedReflexAgent, "Randomized Reflex Agent")
print('First 10 scores:', scores[:10])

Randomized Reflex Agent had an average performance of 13722.33 over 100 trials
First 10 scores: [13590, 13777, 13917, 13609, 13765, 13799, 13725, 13866, 13928, 13806]


_Sample Environment 2_

In [17]:
geography2 = [['AA', 'AB', 'AC', 'AD', 'AE'],
              ['BA', None, None, 'BD', 'BE'],
              ['CA', None, None, 'CD', None],
              ['DA', 'DB', 'DC', 'DD', 'DE']]

env2 = UnknownVacuumWorld(geography=geography2)
env2.display_geography()

scores = run_trials(100, env2, RandomizedReflexAgent, "Randomized Reflex Agent")
print('First 10 scores:', scores[:10], '\n')

scores = run_trials(100, env2, SimpleReflexAgentUnknown, 
                    "Simple Reflex Agent (Traveling Left)", direction='Left')
print('First 10 scores:', scores[:10])

|=||=||=||=||=|
|=|      |=||=|
|=|      |=|   
|=||=||=||=||=|
Randomized Reflex Agent had an average performance of 14518.75 over 100 trials
First 10 scores: [14034, 14667, 14309, 14407, 14455, 14251, 14623, 14714, 14469, 14407] 

Simple Reflex Agent (Traveling Left) had an average performance of 8348.60 over 100 trials
First 10 scores: [10997, 8000, 10000, 11000, 9988, 9000, 6996, 7000, 10999, 4000]


_Sample Environment 3_

In [18]:
geography3 = [['AA', 'AB', 'AC', None, None],
              [None, None, 'BC', None, None],
              [None, None, 'CC', 'CD', 'CE']]

env3 = UnknownVacuumWorld(geography=geography3)
env3.display_geography()

scores = run_trials(100, env3, RandomizedReflexAgent, "Randomized Reflex Agent")
print('First 10 scores:', scores[:10], '\n')

scores = run_trials(100, env3, SimpleReflexAgentUnknown, 
                    "Simple Reflex Agent (Traveling Left)", direction='Left')
print('First 10 scores:', scores[:10])

|=||=||=|      
      |=|      
      |=||=||=|
Randomized Reflex Agent had an average performance of 6867.22 over 100 trials
First 10 scores: [6973, 6997, 6992, 6935, 6962, 6905, 6905, 6878, 6886, 6755] 

Simple Reflex Agent (Traveling Left) had an average performance of 4529.11 over 100 trials
First 10 scores: [4000, 3000, 3999, 5999, 5000, 2000, 3999, 3000, 4000, 3999]


_Sample Environment 4_

In [19]:
geography4 = [['A', 'B', 'C', 'D', 'E', 'F', 'G']]

env4 = UnknownVacuumWorld(geography=geography4)
env4.display_geography()

scores = run_trials(100, env4, RandomizedReflexAgent, "Randomized Reflex Agent")
print('First 10 scores:', scores[:10], '\n')

scores = run_trials(100, env4, SimpleReflexAgentUnknown, 
                    "Simple Reflex Agent (Traveling Left)", direction='Left')
print('First 10 scores:', scores[:10])

|=||=||=||=||=||=||=|
Randomized Reflex Agent had an average performance of 6892.18 over 100 trials
First 10 scores: [6951, 6817, 6768, 6408, 6853, 7000, 6815, 6949, 6952, 6813] 

Simple Reflex Agent (Traveling Left) had an average performance of 5604.78 over 100 trials
First 10 scores: [6982, 6994, 6992, 4998, 5000, 5996, 5000, 6998, 4991, 5998]


_Sample Environment 5_

In [20]:
geography5 = [[None, 'AB', 'AC', 'AD', None],
              ['BA', 'BB', None, 'BD', 'BE'],
              ['CA', 'CB', None, 'CD', 'CE'],
              [None, 'DB', 'DC', 'DD', None]]

env5 = UnknownVacuumWorld(geography=geography5)
env5.display_geography()

scores = run_trials(100, env5, RandomizedReflexAgent, "Randomized Reflex Agent")
print('First 10 scores:', scores[:10], '\n')

scores = run_trials(100, env5, SimpleReflexAgentUnknown, 
                    "Simple Reflex Agent (Traveling Up)", direction='Up')
print('First 10 scores:', scores[:10])

   |=||=||=|   
|=||=|   |=||=|
|=||=|   |=||=|
   |=||=||=|   
Randomized Reflex Agent had an average performance of 13661.94 over 100 trials
First 10 scores: [13458, 13895, 13774, 13439, 13246, 13601, 13745, 13790, 13872, 13677] 

Simple Reflex Agent (Traveling Up) had an average performance of 7928.80 over 100 trials
First 10 scores: [8000, 6999, 6999, 5000, 6000, 7999, 7999, 9988, 10000, 10997]


__c. Can you design an environment in which your randomized agent will perform poorly? Show your results.__

If the geography of the environment contains bottlenecks,then a randomized agent may have a tough time navigating through them by chance and reaching all the dirty squares. To demonstrate that a bottleneck is responsible for lowered performance, I will also show agent performance on an environment with the same total number of squares, but no bottleneck.

In [21]:
bottle_geo = [['AA', 'AB', 'AC', 'AD', 'AE'],
              ['BA', 'BB', 'BC', 'BD', 'BE'],
              [None, None, 'CC', None, None],
              ['DA', 'DB', 'DC', 'DD', 'DE'],
              ['EA', 'EB', 'EC', 'ED', 'EE']]

bottle_env = UnknownVacuumWorld(geography=bottle_geo, dirt_init='dirty')
bottle_env.display_geography()

scores = run_trials(100, bottle_env, RandomizedReflexAgent, "Randomized Reflex Agent")
print('First 10 scores:', scores[:10])

|=||=||=||=||=|
|=||=||=||=||=|
      |=|      
|=||=||=||=||=|
|=||=||=||=||=|
Randomized Reflex Agent had an average performance of 18695.43 over 100 trials
First 10 scores: [18943, 16146, 16753, 19689, 18644, 19353, 20007, 20100, 17670, 18446]


In [22]:
non_bottle_geo = [['AA', 'AB', 'AC', 'AD', 'AE'],
                  ['BA', 'BB', 'BC', 'BD', 'BE'],
                  ['CA', 'CB', 'CC', 'CD', 'CE'],
                  ['DA', 'DB', 'DC', 'DD', 'DE'],
                  [None, None, 'EC', None, None]]

non_bottle_env = UnknownVacuumWorld(geography=non_bottle_geo, dirt_init='dirty')
non_bottle_env.display_geography()

scores = run_trials(100, non_bottle_env, RandomizedReflexAgent, "Randomized Reflex Agent")
print('First 10 scores:', scores[:10])

|=||=||=||=||=|
|=||=||=||=||=|
|=||=||=||=||=|
|=||=||=||=||=|
      |=|      
Randomized Reflex Agent had an average performance of 19745.21 over 100 trials
First 10 scores: [20074, 19706, 19833, 19500, 19823, 20082, 20319, 19825, 19738, 19551]


__d. Can a reflex agent with state outperform a simple reflex agent? Design such an agent and measure its performance on several environments. Can you design a rational agent of this type?__

A reflex agent with state can keep track of the squares it has visited, and therefore should be able to implement basic navigation strategies to explore the space, such as zigzagging.

The problem of mapping out and keeping track of one's own position within an environment is known as Simultaneous Localization and Mapping (SLAM). [As far as I am aware](https://www.quora.com/Is-the-SLAM-simultaneous-localization-and-mapping-problem-in-robot-navigation-solved-for-dynamic-changing-environments), it is not considered a "solved" problem even for static environments.

A rational agent would need to have a prior belief over the set of possible grid topologies (boundaries and obstacles) in order to be able to calculate its expected performance and choose the strategy for traversing the unknown space. Given some prior, designing a rational reflex agent with state may be possible, but would likely be intractable.

I implement an agent that explores in a depth-first manner, going in one direction and adding to a queue of explored squares and the directions unexplored from there. When it hits an obstacle, it changes directions. When all directions away from a square are explored, it is taken off the list. When an agent has no directions to explore from its current square, it backtracks to the most recent square with adjacent directions left unexplored.

In [23]:
from vacuum_cleaner_world.agents import StatefulReflexAgentUnknown

In [24]:
env.display_geography()

|=||=||=|   
|=||=||=||=|
|=|   |=||=|
|=||=||=||=|


In [40]:
scores = run_trials(100, non_bottle_env, StatefulReflexAgentUnknown, "Stateful Reflex Agent")
print('First 10 scores:', scores[:10])

Stateful Reflex Agent had an average performance of 2604.28 over 100 trials
First 10 scores: [3988, 1000, 3988, 3988, 1998, 1000, 1000, 2994, 3988, 3988]


In [26]:
thing

[]